# Build your own OpenAI Agent

With the [new OpenAI API](https://openai.com/blog/function-calling-and-other-api-updates) that supports function calling, it's never been easier to build your own agent!

In this notebook tutorial, we showcase how to write your own OpenAI agent in **under 50 lines of code**! It is minimal, yet feature complete (with ability to carry on a conversation and use tools).

## Initial Setup 

Let's start by importing some simple building blocks.  

The main thing we need is:
1. the OpenAI API (we will use langchain's ChatOpenAI wrapper for convienience here.)
2. a place to keep conversation history 
3. a definition for tools that our agent can use.

In [1]:
import json
from typing import Sequence, List

from llama_index.llms import OpenAI, ChatMessage
from llama_index.tools import BaseTool, FunctionTool

import nest_asyncio
nest_asyncio.apply()

Let's define some very simple calculator tools for our agent.

In [2]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [3]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

add_tool = FunctionTool.from_defaults(fn=add)

## Agent Definition

Now, we define our agent that's capable of holding a conversation and calling tools in **under 50 lines of code**.

The meat of the agent logic is in the `chat` method. At a high-level, there are 3 steps:
1. Call OpenAI to decide which tool (if any) to call and with what arguments.
2. Call the tool with the arguments to obtain an output
3. Call OpenAI to synthesize a response from the conversation context and the tool output.

The `reset` method simply resets the conversation context, so we can start another conversation.

In [14]:
class YourOpenAIAgent:
    def __init__(
        self,
        tools: Sequence[BaseTool] = [],
        llm: OpenAI = OpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
        chat_history: List[ChatMessage] = [],
    ) -> None:
        self._llm = llm
        self._tools = {tool.metadata.name: tool for tool in tools}
        self._chat_history = chat_history

    def reset(self) -> None:
        self._chat_history = []

    def chat(self, message: str) -> str:
        chat_history = self._chat_history
        chat_history.append(ChatMessage(role='user', content=message))
        functions = [tool.metadata.to_openai_function() for _, tool in self._tools.items()]

        ai_message = self._llm.chat(chat_history, functions=functions).message
        chat_history.append(ai_message)

        function_call = ai_message.additional_kwargs.get("function_call", None)
        if function_call is not None:
            function_message = self._call_function(function_call)
            chat_history.append(function_message)
            ai_message = self._llm.chat(chat_history).message
            chat_history.append(ai_message)

        return ai_message.content

    def _call_function(self, function_call: dict) -> ChatMessage:
        tool = self._tools[function_call["name"]]
        output = tool(**json.loads(function_call["arguments"]))
        return ChatMessage(
            name=function_call["name"],
            content=str(output), 
            role='function',
            additional_kwargs={
                "name": function_call["name"]
            }
        )

## Let's Try It Out!

In [15]:
agent = YourOpenAIAgent(tools=[multiply_tool, add_tool])

In [16]:
agent.chat('Hi')

'Hello! How can I assist you today?'

In [17]:
agent.chat('What is 2123 * 215123')

'The product of 2123 multiplied by 215123 is 456,706,129.'

## Our (Slightly Better) `OpenAIAgent` Implementation 

We provide a (slightly better) `OpenAIAgent` implementation in LlamaIndex, which you can directly use as follows.  

In comparison to the simplified version above:
* it implements the `BaseChatEngine` and `BaseQueryEngine` interface, so you can more seamlessly use it in the LlamaIndex framework. 
* it supports multiple function calls per conversation turn
* it supports streaming
* it supports async endpoints
* it supports callback and tracing

In [4]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI

In [5]:
llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = OpenAIAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)

### Chat

In [6]:
response = agent.chat("What is 121 * 3?")
print(str(response))

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363
121 multiplied by 3 is equal to 363.


### Async Chat

In [7]:
response = await agent.achat("What is 121 * 3?")
print(str(response))

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363
121 multiplied by 3 is equal to 363.


### Streaming Chat
Here, every LLM response is returned as a generator. You can stream every incremental step, or only the last response.

In [8]:
agent_stream = agent.stream_chat("What is 121 * 3?")
for response in agent_stream:
    response_gen = response.response_gen
    # NOTE: here, we skip any intermediate steps and wait until the last response 
    #
    # for token in response_gen:
    #     print(token, end="")

for token in response.response_gen:
    print(token, end="")

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363
121 multiplied by 3 is equal to 363.

### Async Streaming Chat

In [9]:
chat_gen = agent.astream_chat("What is 121 * 3?")

async for chat_stream in chat_gen:
    for token in chat_stream.response_gen:
        print(token, end="")

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363
121 multiplied by 3 is equal to 363.